<a href="https://colab.research.google.com/github/nedazar/NLP-Assignment-2---Group-15/blob/main/Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
from google.colab import drive, files
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, average_precision_score, recall_score, classification_report
import seaborn as sn
import matplotlib.pyplot as plt
import torch
from google.colab import drive
from transformers import BertTokenizerFast
from statistics import mean

In [ ]:
# load bert model
model = ClassificationModel('bert','bert-base-cased')

# mount file path to google drive
drive.mount('/content/drive')

In [4]:
## LOAD TRAINNING DATA
olid_train = pd.read_csv('/content/drive/MyDrive/VU/NLPT/Assignment 2/olid-train.csv')
olid_train = olid_train.drop(columns = ['id'])
olid_train.head()

,text,labels
0,@USER She should ask a few native Americans wh...,1
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,1
2,Amazon is investigating Chinese employees who ...,0
3,"@USER Someone should'veTaken"" this piece of sh...",1
4,@USER @USER Obama wanted liberals &amp; illega...,0


In [5]:
# LOAD TEST DATA
olid_test = pd.read_csv('/content/drive/MyDrive/VU/NLPT/Assignment 2/olid-test.csv')
olid_test = olid_test.drop(columns = ['id'])
olid_test.head()

,text,labels
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,"#ConstitutionDay is revered by Conservatives, ...",0
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,#Watching #Boomer getting the news that she is...,0
4,#NoPasaran: Unity demo to oppose the far-right...,1


In [6]:
# Train the model
""" this is replaced by load de saved trained model from drive
 model.train_model(olid_train)

#Saving fine-tuned BERT model in drive so we dont have to fine-tune it every time

torch.save(model, '/content/drive/MyDrive/VU/NLPT/Assignment 2/saved_model') 
"""

# Load saved TRAINED model
model = torch.load('/content/drive/MyDrive/VU/NLPT/Assignment 2/saved_model')

In [7]:
# Downloading the BERTmodel tokenizer

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [8]:
# 4.A  NUMBER OF TOKENS
token_counter = 0

for text in olid_train['text']:
  token_counter += len(tokenizer.tokenize(text))
  
print("Number of tokens: ",token_counter)

Number of tokens:  478955


In [9]:
#4.A  and B 
"""
  returns token divided in subwords by tokenizer """
def find_subword(tokens):
  subwords = []

  for token in tokens:
    if '##' in token:
      subwords.append(token.replace('##','').strip())
    else:
      return subwords
  
  return subwords

counts_tokens = []
token_subwords_length = []

for text in olid_train['text']:
  tokens = tokenizer.tokenize(text)
  count, i = (0, 0)

  while i < len(tokens):
    token = tokens[i]

    subwords_token = [token] + find_subword(tokens[i+1:])
    token_subwords_length.append(len(subwords_token))

    if len(subwords_token) > 1:
      i += len(subwords_token)
      count += 1
      #print("Token divided into subword : ",''.join(subwords_token))
    else:
      i += 1
  counts_tokens.append(count)

print("\nNumber of tokens divided into subwords :",sum(counts_tokens))
print("Average number of subwords per token: ",round(mean(token_subwords_length),2))


Number of tokens divided into subwords : 67045
Average number of subwords per token:  1.23


In [10]:
# 4.D  
vocab = model.tokenizer.vocab
keys = list(vocab.keys())

longest_subword= max(keys,key=len)

print("The longest subword of vocabulary is: ",longest_subword, "and the length is: ",len(longest_subword))

The longest subword of vocabulary is:  telecommunications and the length is:  18
